In [1]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

In [2]:
from dotenv import load_dotenv
load_dotenv('../.env')

True

In [15]:
from io import StringIO
from contextlib import redirect_stdout

def execPythonCommand(command: str) -> str:
    command = command.strip().strip('`').strip().replace("\\n", "\n")

    # be evil
    # command = command.replace('5','7')

    answer = input("\nRun the following code? (y/n) \n> " + command)
    if answer.lower() != 'y':
        return f"The operator denied your request to run the specified code with the following response: {answer}"

    outString = StringIO()
    with redirect_stdout(outString):
        exec(command)
    return outString.getvalue()    

exec_python_tool = Tool(
    name = "Exec",
    func = execPythonCommand,
    description = "Does an exec of the specified python code and returns the output. Use if you need to execute python code to complete a task."
)

def helpCommand(command:str) -> str:
    answer = input("\n" + command)
    return answer

help_tool = Tool(
    name = "Help",
    func = helpCommand,
    description= "Asks the human operator for advice or suggestions. The input should be a question or a description of a problem. Use only as a last resort if attempts to solve the problem yourself have failed."
)

tools = [exec_python_tool, help_tool] + load_tools(["google-search"])

In [16]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)
agent = initialize_agent(tools, llm,agent="zero-shot-react-description", verbose=True, max_iterations=10)

In [17]:
question1 = "Give me a short bio of ilya sutskever"

with get_openai_callback() as cb:
    response = agent.run(question1)
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")



> Entering new AgentExecutor chain...
I'm not sure who Ilya Sutskever is, so I should probably use Google Search to find out more about him.
Action: Google Search
Action Input: "Ilya Sutskever bio"
Observation: Ilya Sutskever FRS (Hebrew: איליה סוצקבר; born 1985/86) is an Israeli-Canadian computer scientist working in machine learning, who co-founded and serves as ... Co-founder and Chief Scientist of OpenAI. I spent three wonderful years as a Research Scientist at the Google Brain Team. Before that, I was a co-founder of ... View Ilya Sutskever's profile on LinkedIn, the world's largest professional community. Ilya has 4 jobs listed on their profile. See the complete profile on ... Ilya Sutskever, the co-inventor of AlexNet, is also the inventor of sequence of sequence learning (as the first author). He has led the revolution in both ... Ilya Sutskever focuses on Artificial intelligence, Algorithm, Task, Language model and Contrast. As part of his studies on Artificial intelligence,

In [ ]:
fizzbuzz = """```
for i in range(1, 21):
    if i % 3 == 0 and i % 5 == 0:
        print("FizzBuzz")
    elif i % 3 == 0:
        print("Fizz")
    elif i % 5 == 0:
        print("Buzz")
    else:
        print(i)
```
"""

processed = fizzbuzz.strip().strip('`').strip()
exec(processed)

1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
FizzBuzz
16
17
Fizz
19
Buzz
